# (실습-6) Vector 유사도 검색 실습 - Faiss

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 Faiss를 이용하여 vector 유사도 검색을 실험해 본다. <br>
  Elasticsearch와 비교하여 사용성이 어떻게 다른지 확인한다. <br>
2) 수강 목표
  * Faiss를 이용하여 vector 유사도 검색을 할 수 있다.
  * Faiss API를 사용할 수 있다.
  * Elasticsearch 대비 차이점을 이해한다.

### 실습 목차
* 1. 데이터 전처리
* 2. 색인 및 검색

### 데이터셋 개요
* 데이터셋: wikimedia kowiki
* 데이터셋 개요 : wikimedia에서 제공하는 한국어 데이터셋

### 환경 설정
Faiss 패키지를 설치한다. <br>
Sentence-transformers 패키지를 설치한다.

In [1]:
# GPU를 사용하는 버전의 Faiss Python 패키지 설치
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00


In [2]:
# 임베딩 생성을 위한 벡터 인코더 설치
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=45fda24dc625ef8326df115d0eed978421309ae29eac3b56d9edf7b6076c5436
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


## 1. 데이터 전처리

In [3]:
# 위키미디어로부터 kowiki 데이터를 다운로드 받음
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
# 위키데이터의 노이즈를 제거하고 json 형태로 반환하는 코드를 참조
!git clone https://github.com/attardi/wikiextractor.git
# 다운로드 받은 샘플 위키 데이터를 전처리하여 검색의 입력으로 사용
# 결과는 elastic 폴더에 'extract_result/AA,AB,AC.../wiki_00..99'라는 새로운 폴더에 저장된다.(용량이 비슷하게 나눠서 저장됨)
# 변환결과 wiki_00 파일의 내용 샘플  {"id": "5", "revid": "641228", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "\uc9c0\...\ud130", "text": "\uc81c\...\ub2e4."}
!python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles1.xml-p1p82407.bz2 --json -o extract_result

--2024-01-24 05:39:33--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74589104 (71M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’

kowiki-latest-pages 100%[===================>]  71.13M  4.17MB/s    in 18s     

2024-01-24 05:39:52 (4.06 MB/s) - ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’ saved [74589104/74589104]

Cloning into 'wikiextractor'...
remote: Enumerating objects: 771, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 771 (delta 17), reused 21 (delta 14), pack-reused 741
Receiving objects: 100% (771/771), 1.31 MiB | 22.77 MiB/s, done.
Resolving deltas: 100% (450/450), done.
INFO: Pr

In [4]:
import json
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("hunkim/sentence-transformer-klue")

def get_embedding(sentences):
    # 입력 문장을 인코딩하여 임베딩을 얻음
    return model.encode(sentences)

wiki_dump_json_file = '/content/extract_result/AA/wiki_00'
# 'wiki_dump_json_file'에 있는 JSON 파일 읽어들여 index_docs에 저장
index_docs = []

for line in open(wiki_dump_json_file, encoding="utf-8"):
    # JSON 데이터를 읽어들여 파이썬 딕셔너리로 변환
    json_data = json.loads(line)

    # text와 title에 대한 임베딩을 계산하여 해당 필드에 추가
    json_data['embeddings_title'] = get_embedding(json_data['title']).tolist()
    json_data['embeddings_text'] = get_embedding(json_data['text']).tolist()

    # 색인할 문서 목록에 추가
    index_docs.append(json_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [5]:
# Faiss는 특정 임베딩 필드 하나만 저장할 수 있기 때문에 아래와 같이 임베딩 필드만 추출
title_docs = [item["embeddings_title"] for item in index_docs]
text_docs = [item["embeddings_text"] for item in index_docs]

## 2. 색인 및 검색

In [6]:
# 테스트로 가장 간단한 Index type인 FlatL2를 사용

import faiss
import numpy as np

index_title = faiss.IndexFlatL2(768)
index_title.add(np.array(title_docs).astype('float32'))

index_text = faiss.IndexFlatL2(768)
index_text.add(np.array(text_docs).astype('float32'))

In [7]:
# 결과로 offset과 score가 나오는데 원본 문서와의 매칭은 offset 값을 이용

query_str = "문재인의 친구"
query_emb = get_embedding([query_str])
scores, offsets = index_title.search(query_emb, 5)

for i,offset in enumerate(offsets[0]):
    print(f'{offset}, {scores[0][i]} {index_docs[offset]}')

38, 134.9847869873047 {'id': '85', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 장관을 역임했다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시행한 것, 대

In [8]:
# 다른 질의로 한번더 실행

query_str = "대한민국 16대 대통령이 누구야?"
query_emb = get_embedding([query_str])
scores, offsets = index_title.search(query_emb, 5)

for i,offset in enumerate(offsets[0]):
    print(f'{offset}, {scores[0][i]} {index_docs[offset]}')

10, 58.793067932128906 {'id': '34', 'revid': '36250640', 'url': 'https://ko.wikipedia.org/wiki?curid=34', 'title': '대한민국 제16대 대통령 선거', 'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 해양수산부 장관 출신 노무현 새천년민주당 후보의 양강 구도로 진행되었다.\n대선 재수생인 이회창 후보는 경험이나 세력 면에서 노무현 후보보다 대권 고지에 좀 더 유리할 것으로 점쳐졌으나, 이전 대선부터 불거진 이회창 후보의 두 아들의 병역기피 논란, 노사모를 비롯한 네티즌들의 열성적인 노무현 지지, 정몽준 후보와의 단일화 성공 등에 힘입어 노무현 후보가 당선되었다.\n선거 정보.\n선거권.\n만 20세 이상의 대한민국 국민은 선거권이 있었다. 즉, 1982년 12월 19일 이전에 태어난 사람은 투표를 할 자격이 있었다.\n피선거권.\n만 40세 이상의 대한민국 국민은 피선거권을 가졌다. 즉, 1962년 12월 19일 이전에 태어난 사람은 후보자가 될 자격이 있었다.\n후보.\n새천년민주당.\n새천년민주당은 3월 9일부터 4월 27일까지 한국 정당 역사상 최초로 국민 참여 경선을 실시하고 과반 득표자인 노무현 전, 판사, 변호사, 해양수산부 장관을 대통령 후보로 선출하였다.\n한나라당.\n한나라당은 4월 13일부터 5월 9일까지 국민 참여 경선을 실시하고 최다 득표자인 이회창 전 당 총재를 대통령 후보로 선출하였다.\n민주노동당.\n민주노동당은 9월 8일 당원들에 의한 단일 후보 찬반 투표를 통해 권영길 당 대표를 대통령 후보로 선출하였다.\n국민통합21.\n통

In [9]:
# text 필드에서 검색

query_str = "문재인의 친구"
query_emb = get_embedding([query_str])
scores, offsets = index_text.search(query_emb, 5)

for i,offset in enumerate(offsets[0]):
    print(f'{offset}, {scores[0][i]} {index_docs[offset]}')

38, 160.76795959472656 {'id': '85', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 장관을 역임했다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시행한 것, 

In [10]:
# 다른 질의로 한번더 실행

query_str = "대한민국 16대 대통령이 누구야?"
query_emb = get_embedding([query_str])
scores, offsets = index_text.search(query_emb, 5)

for i,offset in enumerate(offsets[0]):
    print(f'{offset}, {scores[0][i]} {index_docs[offset]}')

10, 152.64010620117188 {'id': '34', 'revid': '36250640', 'url': 'https://ko.wikipedia.org/wiki?curid=34', 'title': '대한민국 제16대 대통령 선거', 'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 해양수산부 장관 출신 노무현 새천년민주당 후보의 양강 구도로 진행되었다.\n대선 재수생인 이회창 후보는 경험이나 세력 면에서 노무현 후보보다 대권 고지에 좀 더 유리할 것으로 점쳐졌으나, 이전 대선부터 불거진 이회창 후보의 두 아들의 병역기피 논란, 노사모를 비롯한 네티즌들의 열성적인 노무현 지지, 정몽준 후보와의 단일화 성공 등에 힘입어 노무현 후보가 당선되었다.\n선거 정보.\n선거권.\n만 20세 이상의 대한민국 국민은 선거권이 있었다. 즉, 1982년 12월 19일 이전에 태어난 사람은 투표를 할 자격이 있었다.\n피선거권.\n만 40세 이상의 대한민국 국민은 피선거권을 가졌다. 즉, 1962년 12월 19일 이전에 태어난 사람은 후보자가 될 자격이 있었다.\n후보.\n새천년민주당.\n새천년민주당은 3월 9일부터 4월 27일까지 한국 정당 역사상 최초로 국민 참여 경선을 실시하고 과반 득표자인 노무현 전, 판사, 변호사, 해양수산부 장관을 대통령 후보로 선출하였다.\n한나라당.\n한나라당은 4월 13일부터 5월 9일까지 국민 참여 경선을 실시하고 최다 득표자인 이회창 전 당 총재를 대통령 후보로 선출하였다.\n민주노동당.\n민주노동당은 9월 8일 당원들에 의한 단일 후보 찬반 투표를 통해 권영길 당 대표를 대통령 후보로 선출하였다.\n국민통합21.\n통

In [11]:
# 오탈자 확인

query_str = "대한민국 16대 대통려이 누구야?"
query_emb = get_embedding([query_str])
scores, offsets = index_text.search(query_emb, 5)

for i,offset in enumerate(offsets[0]):
    print(f'{offset}, {scores[0][i]} {index_docs[offset]}')

10, 151.15846252441406 {'id': '34', 'revid': '36250640', 'url': 'https://ko.wikipedia.org/wiki?curid=34', 'title': '대한민국 제16대 대통령 선거', 'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 해양수산부 장관 출신 노무현 새천년민주당 후보의 양강 구도로 진행되었다.\n대선 재수생인 이회창 후보는 경험이나 세력 면에서 노무현 후보보다 대권 고지에 좀 더 유리할 것으로 점쳐졌으나, 이전 대선부터 불거진 이회창 후보의 두 아들의 병역기피 논란, 노사모를 비롯한 네티즌들의 열성적인 노무현 지지, 정몽준 후보와의 단일화 성공 등에 힘입어 노무현 후보가 당선되었다.\n선거 정보.\n선거권.\n만 20세 이상의 대한민국 국민은 선거권이 있었다. 즉, 1982년 12월 19일 이전에 태어난 사람은 투표를 할 자격이 있었다.\n피선거권.\n만 40세 이상의 대한민국 국민은 피선거권을 가졌다. 즉, 1962년 12월 19일 이전에 태어난 사람은 후보자가 될 자격이 있었다.\n후보.\n새천년민주당.\n새천년민주당은 3월 9일부터 4월 27일까지 한국 정당 역사상 최초로 국민 참여 경선을 실시하고 과반 득표자인 노무현 전, 판사, 변호사, 해양수산부 장관을 대통령 후보로 선출하였다.\n한나라당.\n한나라당은 4월 13일부터 5월 9일까지 국민 참여 경선을 실시하고 최다 득표자인 이회창 전 당 총재를 대통령 후보로 선출하였다.\n민주노동당.\n민주노동당은 9월 8일 당원들에 의한 단일 후보 찬반 투표를 통해 권영길 당 대표를 대통령 후보로 선출하였다.\n국민통합21.\n통

#Reference

## Required Package

sentence_transformers=2.2.2 <br>
faiss-gpu==1.7.2
